In [ ]:
# Load necessar libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
# Load the heart disease data
data = pd.read_csv('..\data\heart_disease\heart_disease_uci.csv')
data.head()


In [ ]:
#Converting num to binary
data['num'] = data['num'].apply(lambda x: 1 if x>0 else 0)
data.head()

In [ ]:
#Splitting the data into features and target
features = data.iloc[:,:-1]
target = data.iloc[:,-1]

features.head()

In [ ]:
#Finding categorical columns and using ordinal encoding
categorical_columns = features.select_dtypes(include='object').columns
categorical_columns


In [ ]:
# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, stratify=target, random_state=42)


In [ ]:
# Convert to DataFrame for easier handling
train_data = pd.DataFrame(X_train)
train_data['target'] = y_train.values

test_data = pd.DataFrame(X_test)
test_data['target'] = y_test.values



In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the Mistral model and tokenizer from Hugging Face
model_name = "mistralai/mistral-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, use_auth_token=True)


In [ ]:
# Tokenize the training data
def tokenize_row(row, tokenizer):
    row_str = " ".join(map(str, row.values))
    inputs = tokenizer(row_str, padding='max_length', truncation=True, return_tensors="pt")
    return inputs

train_inputs = [tokenize_row(row, tokenizer) for _, row in train_data.iterrows()]
test_inputs = [tokenize_row(row, tokenizer) for _, row in test_data.iterrows()]


In [ ]:
# Tokenize the training data
def tokenize_row(row, tokenizer):
    row_str = " ".join(map(str, row.values))
    inputs = tokenizer(row_str, padding='max_length', truncation=True, return_tensors="pt")
    return inputs

train_inputs = [tokenize_row(row, tokenizer) for _, row in train_data.iterrows()]
test_inputs = [tokenize_row(row, tokenizer) for _, row in test_data.iterrows()]


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class HeartDiseaseDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        input_ids = self.inputs[idx]['input_ids'].squeeze(0)
        attention_mask = self.inputs[idx]['attention_mask'].squeeze(0)
        label = torch.tensor(self.labels.iloc[idx])
        return input_ids, attention_mask, label

# Create dataset and dataloader
train_dataset = HeartDiseaseDataset(train_inputs, y_train)
test_dataset = HeartDiseaseDataset(test_inputs, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


In [ ]:
from transformers import AdamW

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training function
def train(model, train_loader, optimizer):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    return total_loss / len(train_loader)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    avg_loss = train(model, train_loader, optimizer)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')


In [ ]:
def evaluate(model, test_loader):
    model.eval()
    total_correct = 0
    total_examples = 0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predictions.cpu().numpy())
            total_correct += (predictions == labels).sum().item()
            total_examples += labels.size(0)
    accuracy = total_correct / total_examples
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    return accuracy, precision, recall, f1

# Evaluate the Mistral model
mistral_accuracy, mistral_precision, mistral_recall, mistral_f1 = evaluate(model, test_loader)
print(f'Mistral Model - Accuracy: {mistral_accuracy:.4f}, Precision: {mistral_precision:.4f}, Recall: {mistral_recall:.4f}, F1-Score: {mistral_f1:.4f}')
